In [33]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from substitutions import appos
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
import numpy as np
import contractions
import copy

[nltk_data] Downloading package stopwords to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
data_dir = '../Humor,Hist,Media,Food'
file_names = os.listdir(data_dir)
file_paths = [(data_dir + '/' + fname) for fname in file_names]
docID_to_doc = {}
for i in range(len(file_names)):
    docID_to_doc[i] = file_names[i]

In [54]:

def read_files(fpaths):
    file_tokens = []
    for fpath in fpaths:
        f = open(fpath, 'r', encoding='utf-8', errors='replace')
        ftxt_unprocessed = f.read()
        # print(ftxt_unprocessed)
        ftoks = preprocess_file(ftxt_unprocessed)
        file_tokens.append(ftoks)
    return file_tokens

def isValidTok(tok):
    if((tok not in string.punctuation) and (tok.isnumeric() == False) and (sum([0 if ch in string.punctuation else 1 for ch in tok]) >= 1)):
        return True
    return False 
def check_alnum(tok):
    tok = ''.join(x for x in tok if x.isalnum() == True)
    return tok
def remove_punct(tok):
    punctuations = string.punctuation
    tok = ''.join(x for x in tok if x not in punctuations)
    return tok

def remove_space(tok):
    tok = ''.join(x for x in tok if x != ' ')
    return tok

def preprocess_file(file_text):
    '''
        Preprocess the file text
    '''
    #converting text to lowercase
    text = file_text.lower()
    
    # print(f"Original tokens : \n{word_tokenize(text)}\n")
    # print("\n-------------------------\n")

    #Fixing the contractions
    text = contractions.fix(text)

    #omitting all non-alphanumeric characters
    # text = ''.join([x if (x.isalnum() == True or x == " ") else " " for x in text ])

    #word tokenization
    all_toks = word_tokenize(text)
    all_unique_toks = set(all_toks)

    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    # print(f"All unique tokens :\n{all_unique_toks}\n")
    # print("\n-------------------------\n")
    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]
    # print(f"After removing stopwords :\n{file_toks}\n")
    # print("\n-------------------------\n")
    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print(f"After removing puncts :\n{toks_no_punct}\n")
    # print("\n-------------------------\n")

    #removing spaces in any remain & check if it is a number
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            cleaned_toks.append(ctok)
    cleaned_toks = list(set(cleaned_toks))
    # print(f"After removing spaces and removing numbers :\n{cleaned_toks}\n")
    # print("\n-------------------------\n")
    #stemming tokens
    # stemmer = PorterStemmer()
    # stemmed_toks = set()

    # for tok in cleaned_toks:
    #     stok = stemmer.stem(tok)
    #     if(stok != "" or (stok not in stop_words)): #consider non-blank tokens and the ones which have length greater than 1
    #         stemmed_toks.add(stok)
    # print(f"After stemming and removing single letters :\n{stemmed_toks}\n")
    # print("\n-------------------------\n")
    final_tokens = [tok for tok in cleaned_toks]

    return final_tokens

def preprocess_query(query_text):

    text = query_text.lower()
    
    # print(f"Original Query Tokens : \n{word_tokenize(text)}\n")
    # print("\n-------------------------\n")
    
    #Fixing the contractions
    text = contractions.fix(text)

    #omitting all non-alphanumeric characters
    # text = ''.join([x if (x.isalnum() == True or x == " ") else " " for x in text ])

    #word tokenization
    all_toks = word_tokenize(text)
    all_unique_toks = []
    for tok in all_toks:
        if(tok not in all_unique_toks):
            all_unique_toks.append(tok)

    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    # print(f"All unique tokens :\n{all_unique_toks}\n")
    # print("\n-------------------------\n")
    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]
    # print(f"After removing stopwords :\n{file_toks}\n")
    # print("\n-------------------------\n")
    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print(f"After removing puncts :\n{toks_no_punct}\n")
    # print("\n-------------------------\n")

    #removing spaces in any remain & check if it is a number
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            cleaned_toks.append(ctok)
    
    final_tokens = []
    for tok in cleaned_toks:
        if(tok not in final_tokens):
            final_tokens.append(tok)
    # cleaned_toks = list(set(cleaned_toks))
    # print(f"After removing spaces and removing numbers :\n{cleaned_toks}\n")
    # print("\n-------------------------\n")
    #stemming tokens
    # stemmer = PorterStemmer()
    # stemmed_toks = set()

    # for tok in cleaned_toks:
    #     stok = stemmer.stem(tok)
    #     if(stok != "" and len(stok) > 1 and (stok not in stop_words)): #consider non-blank tokens and the ones which have length greater than 1
    #         stemmed_toks.add(stok)
    # print(f"After stemming and removing single letters :\n{stemmed_toks}\n")
    # print("\n-------------------------\n")
    # final_tokens = [tok for tok in cleaned_toks]

    return final_tokens



    

In [36]:
file_toks = read_files(file_paths)

In [37]:
def create_inverted_index(file_toks):
    inv_index_postings = {}
    inv_index_frequency = {}
    for i in range(len(file_toks)):
        for tok in file_toks[i]:
            if(tok not in inv_index_postings.keys()):
                inv_index_postings[tok] = [i]
                inv_index_frequency[tok] = 1
            else:
                inv_index_postings[tok].append(i)
                inv_index_frequency[tok] += 1
    inv_index_postings = dict(sorted(inv_index_postings.items()))
    terms_list = inv_index_postings.keys()
    for word in terms_list:
        inv_index_postings[word].sort()
    return inv_index_postings, inv_index_frequency

In [38]:
def getDocsFromID(file_names, doc_IDs):
    doc_names = []
    for doc_ID in doc_IDs:
        doc_names.append(file_names[doc_ID])
    return doc_names

In [39]:
def check_equal_arrays(arr1, arr2):

    if(len(arr1) != len(arr2)):
        return False
    
    arr1 = sorted(arr1)
    arr2 = sorted(arr2)
    for i in range(len(arr1)):
        if(arr1[i] != arr2[i]):
            return False
    return True

In [40]:
def retreive_postings(inv_index_postings, term):
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return []
    else:
        posting = inv_index_postings[term]
        return posting

In [41]:
def retreive_frequency(inv_index_frequency, term):
    terms_list = inv_index_frequency.keys()
    if(term not in terms_list):
        return 0
    else:
        freq = inv_index_frequency[term]
        return freq

In [42]:
def retreive_term_info(inv_index_postings, inv_index_frequency, term):
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return [], 0
    else:
        posting = inv_index_postings[term]
        freq = inv_index_frequency[term]
        return posting, freq

In [43]:
def query_AND(posting1, posting2, verbose=False):


    if(posting1 == [] or posting2 == []):
        return 0, 0, []

    ptr1 = 0
    ptr2 = 0
    answer_docID = []

    num_comparisons = 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2)):
        num_comparisons += 1
        # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

        if(posting1[ptr1] == posting2[ptr2]):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1

    num_docs_retreived = len(answer_docID)
    doc_names_retreived = getDocsFromID(file_names, answer_docID)

    if(verbose==True):
        print(f"AND Query: \nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
    
    # temp_verification = list(set(posting1) & set(posting2))
    # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")

    return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def query_OR(posting1, posting2, verbose=False):
    

    if((posting1 == []) and (posting2 == [])):
        return 0, 0, []
    elif((posting1 == []) and (posting2 != [])):
        ans_docs = posting2
        return len(ans_docs), 0, ans_docs
    elif((posting1 != []) and (posting2 == [])):
        ans_docs = posting1
        return len(ans_docs), 0, ans_docs
    else:
        ptr1 = 0
        ptr2 = 0
        answer_docID = []

        num_comparisons = 0
        
        while(ptr1 < len(posting1) and ptr2 < len(posting2)):
            num_comparisons += 1
            # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

            if(posting1[ptr1] == posting2[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif(posting1[ptr1] < posting2[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2[ptr2])
                ptr2 += 1
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        while(ptr2 < len(posting2)):
            answer_docID.append(posting2[ptr2])
            ptr2 += 1


        num_docs_retreived = len(answer_docID)
        doc_names_retreived = getDocsFromID(file_names, answer_docID)

        if(verbose==True):
            print(f"Query OR\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        # temp_verification = list(set(posting1) | set(posting2))
        # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def perform_NOT(posting):

    all_docIDs = [docID for docID in range(len(file_names))]
    if(posting == []):
        return all_docIDs
    print(len(all_docIDs))
    print(posting)
    for docID in posting:
        # print(f"Removing docID : {docID}")
        all_docIDs.remove(docID)

    return all_docIDs

def query_AND_NOT(posting1, posting2, verbose=False):

    if((posting1 == [])):
        return 0, 0, []
    
    posting2_NOT = perform_NOT(posting2)
    ptr1 = 0
    ptr2 = 0
    answer_docID = []

    num_comparisons = 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2_NOT)):
        num_comparisons += 1
        # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")
        if(posting1[ptr1] == posting2_NOT[ptr2]):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2_NOT[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1

    num_docs_retreived = len(answer_docID)
    doc_names_retreived = getDocsFromID(file_names, answer_docID)

    if(verbose==True):
        print(f"Query AND NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
    
    # temp_verification = list(set(posting1) | set(posting2))
    # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")
    return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

def query_OR_NOT(posting1, posting2, verbose=False):
    
    if((posting1 == []) and (posting2 == [])):
        ans_docs = perform_NOT(posting2)
        return len(ans_docs), 0, ans_docs
    elif((posting1 == []) and (posting2 != [])):
        ans_docs = perform_NOT(posting2)
        return len(ans_docs), 0, ans_docs
    elif((posting1 != []) and (posting2 == [])):
        ans_docs = posting1
        return len(ans_docs), 0, ans_docs
    else:
        
        posting2_NOT = perform_NOT(posting2)
        ptr1 = 0
        ptr2 = 0
        answer_docID = []

        num_comparisons = 0

        while(ptr1 < len(posting1) and ptr2 < len(posting2_NOT)):
            num_comparisons += 1
            # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")
            if(posting1[ptr1] == posting2_NOT[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif(posting1[ptr1] < posting2_NOT[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2_NOT[ptr2])
                ptr2 += 1
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        while(ptr2 < len(posting2_NOT)):
            answer_docID.append(posting2_NOT[ptr2])
            ptr2 += 1


        num_docs_retreived = len(answer_docID)
        doc_names_retreived = getDocsFromID(file_names, answer_docID)

        if(verbose==True):
            print(f"Query OR NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID

In [57]:
a = [0, 1, 2, 3, 4, 5]
b = [0, 2, 3, 4, 6]

query_OR_NOT(a, b, verbose=True)

1133
[0, 2, 3, 4, 6]
Query OR NOT
No. of documents retreived: 1132
Minimum number of comparisons: 6
Names of retreived documents: ['1st_aid.txt', 'a-team', 'abbott.txt', 'aboutada.txt', 'acetab1.txt', 'aclamt.txt', 'acronym.lis', 'acronym.txt', 'acronyms.txt', 'adameve.hum', 'adcopy.hum', 'addrmeri.txt', 'admin.txt', 'adrian_e.faq', 'ads.txt', 'adt_miam.txt', 'advrtize.txt', 'aeonint.txt', 'age.txt', 'aggie.txt', 'aids.txt', 'airlines', 'alabama.txt', 'alcatax.txt', 'alcohol.hum', 'alflog.txt', 'allfam.epi', 'allusion', 'all_grai', 'amazing.epi', 'ambrose.bie', 'amchap2.txt', 'analogy.hum', 'aniherb.txt', 'anime.cli', 'anime.lif', 'anim_lif.txt', 'annoy.fascist', 'anorexia.txt', 'answers', 'anthropo.stu', 'antibiot.txt', 'antimead.bev', 'aphrodis.txt', 'appbred.brd', 'appetiz.rcp', 'applepie.des', 'apsaucke.des', 'apsnet.txt', 'arab.dic', 'arcadian.txt', 'argotdic.txt', 'arnold.txt', 'art-fart.hum', 'arthriti.txt', 'ateam.epi', 'atherosc.txt', 'atombomb.hum', 'att.txt', 'aussie.lng', '

(1132,
 6,
 ['1st_aid.txt',
  'a-team',
  'abbott.txt',
  'aboutada.txt',
  'acetab1.txt',
  'aclamt.txt',
  'acronym.lis',
  'acronym.txt',
  'acronyms.txt',
  'adameve.hum',
  'adcopy.hum',
  'addrmeri.txt',
  'admin.txt',
  'adrian_e.faq',
  'ads.txt',
  'adt_miam.txt',
  'advrtize.txt',
  'aeonint.txt',
  'age.txt',
  'aggie.txt',
  'aids.txt',
  'airlines',
  'alabama.txt',
  'alcatax.txt',
  'alcohol.hum',
  'alflog.txt',
  'allfam.epi',
  'allusion',
  'all_grai',
  'amazing.epi',
  'ambrose.bie',
  'amchap2.txt',
  'analogy.hum',
  'aniherb.txt',
  'anime.cli',
  'anime.lif',
  'anim_lif.txt',
  'annoy.fascist',
  'anorexia.txt',
  'answers',
  'anthropo.stu',
  'antibiot.txt',
  'antimead.bev',
  'aphrodis.txt',
  'appbred.brd',
  'appetiz.rcp',
  'applepie.des',
  'apsaucke.des',
  'apsnet.txt',
  'arab.dic',
  'arcadian.txt',
  'argotdic.txt',
  'arnold.txt',
  'art-fart.hum',
  'arthriti.txt',
  'ateam.epi',
  'atherosc.txt',
  'atombomb.hum',
  'att.txt',
  'aussie.lng',
 

In [51]:
def process_ops(ops_string):
    ops = ops_string.split(",")
    ops = [op.strip() for op in ops]
    for op in ops:
        if(not(op == "AND" or op == "OR" or op == "OR NOT" or op == "AND NOT")):
            return []
    return ops

def perform_query(posting1, posting2, op):
    if(op == "AND"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_AND(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "OR"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_OR(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "AND NOT"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_AND_NOT(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    elif(op == "OR NOT"):
        num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = query_OR_NOT(posting1, posting2)
        return num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID
    else:
        print("INVALID OPERATOR")
        return []

def process_query(inv_index, query_tokens, ops):
    # query_tokens = preprocess_query(query_text)
    print(f"Query tokens : {query_tokens}\n")
    print(f"Operators : ")
    n_qtoks = len(query_tokens)
    n_ops = n_qtoks - 1
    # query_operato = copy.deepcopy(ops)
    if(len(ops) > n_ops):
        print(f"Insufficient tokens in the query\n")
        return -1, -1, []
    elif(len(ops) < n_ops):
        print(f"Insufficient operators")
        return -1, -1, []
    else:
        total_comparisons = 0
        if(n_ops == 1):
            p1 = retreive_postings(inv_index, query_tokens[0])
            p2 = retreive_postings(inv_index,query_tokens[1])
            num_docs_retreived, num_comparisons, doc_names_retreived, answer_docID = perform_query(p1, p2, ops[0])
            return num_docs_retreived, num_comparisons, doc_names_retreived
        else:
            p1 = retreive_postings(inv_index, query_tokens[0])
            p2 = retreive_postings(inv_index, query_tokens[1])
            ndocs, total_comparisons, doc_names, ans = perform_query(p1, p2, ops[0])
            qtok_ptr = 2
            for i in range(1, len(ops)):
                posting_tok = retreive_postings(inv_index, query_tokens[qtok_ptr])
                ndocs, cmps, doc_names, ans = perform_query(ans, posting_tok, ops[i])
                total_comparisons += cmps
                qtok_ptr += 1
            num_docs_retreived = len(ans)
            doc_names_retreived = getDocsFromID(file_names, ans)
            return num_docs_retreived, total_comparisons, doc_names_retreived

In [45]:
inverted_index_postings, inverted_index_frequency = create_inverted_index(file_toks)

In [58]:
input_query_sequence = input("Enter the space-separated query terms : ")
input_operator_sequence = input("Enter the comma-separated operator sequence enclosed in '[]' : ")
print(f"Input query sequence : {input_query_sequence}\n")
print(f"Input operator sequence : {input_operator_sequence}\n")
input_operators = process_ops(input_operator_sequence[1:len(input_operator_sequence) - 1])
query_tokens = preprocess_query(input_query_sequence)
print(f"Input operator (array) : {input_operators}\n")
print(f"Query tokens : {query_tokens}\n")
query_result_ndocs, query_result_total_cmps, query_result_doc_names = process_query(inverted_index_postings, query_tokens, input_operators)

if(query_result_ndocs != -1):
    query_generated = query_tokens[0] + " " + input_operators[0] + " " + query_tokens[1] + " "
    ptr_qtoks = 2
    for i in range(1, len(input_operators)):
        query_generated += input_operators[i] + " "
        query_generated += query_tokens[ptr_qtoks] + " "
        ptr_qtoks += 1
    print(f"Query : {query_generated}\n\n")
    print(f"\nNo. of documents retreived: {query_result_ndocs}\nNumber of comparisons: {query_result_total_cmps}\nNames of retreived documents: {query_result_doc_names}")



Input query sequence : happy lion a bad boy

Input operator sequence : [AND, OR, AND NOT]

Input operator (array) : ['AND', 'OR', 'AND NOT']

Query tokens : ['happy', 'lion', 'bad', 'boy']

Query tokens : ['happy', 'lion', 'bad', 'boy']

Operators : 
1133
[1, 9, 15, 20, 23, 27, 28, 30, 52, 56, 83, 88, 92, 115, 116, 128, 129, 130, 135, 175, 182, 195, 213, 216, 223, 235, 251, 252, 256, 264, 268, 270, 278, 286, 288, 301, 316, 318, 324, 326, 346, 348, 349, 350, 351, 352, 354, 378, 379, 392, 395, 401, 406, 414, 416, 417, 418, 420, 421, 424, 425, 426, 437, 445, 450, 475, 480, 501, 505, 514, 529, 533, 537, 547, 548, 557, 581, 584, 586, 600, 607, 608, 610, 618, 622, 630, 635, 641, 645, 650, 663, 671, 672, 682, 691, 716, 717, 718, 724, 752, 753, 756, 782, 785, 786, 796, 804, 812, 813, 814, 815, 817, 822, 824, 835, 838, 839, 853, 855, 889, 909, 910, 911, 916, 921, 928, 933, 956, 960, 977, 980, 981, 985, 1006, 1016, 1017, 1019, 1020, 1028, 1033, 1034, 1035, 1039, 1042, 1044, 1053, 1055, 1057, 106